In [1]:
#Imports

import numpy as np
import matplotlib.pyplot as plt
import hera_cal.abscal as abscal
import uvtools.dspec as dspec 
import itertools
import scipy 
from scipy import signal
import pickle
import copy
from hera_cal.utils import polnum2str, polstr2num, jnum2str, jstr2num
from hera_cal.io import HERAData, HERACal
from hera_cal.io import DataContainer 
from hera_cal import apply_cal
from hera_cal import io
from hera_cal import smooth_cal
from hera_cal import vis_clean
from hera_cal import redcal
from pyuvdata import UVFlag
import glob

In [2]:
#Scale Plot/Figure Sizes

plt.rcParams['figure.figsize'] = 20,10
plt.rcParams['font.size'] = 20

In [3]:
#Discrete Fourier Transform 

def fft(x):
    return np.abs(np.fft.fftshift(np.fft.fft(np.fft.fftshift(x))))

In [4]:
flag_files = sorted(glob.glob('H1C_Flags/*.flags.h5'))
data_file = "./prisim_hera_test/2020-12-07-21-06-34/simdata/all-simvis-noiseless.uvh5"
model_file = "./prisim_hera_test/2020-12-07-19-10-20/simdata/all-simvis-noiseless.uvh5"

In [8]:
abscal.post_redcal_abscal_run(data_file = f'data_{day}.uvh5', redcal_file = f'data_{day}.omni.calfits',
                              model_files = [f"model_{day}.uvh5"], clobber=True, data_solar_horizon=90,
                             model_solar_horizon=90)

The following model files overlap with data files in LST:
model_0.uvh5


Now calibrating nn-polarization...
Selected 171 data baselines and 171 model baselines to load.

    Now calibrating times 2457053.0807914445 through 2457053.2052039444...


KeyboardInterrupt: 

In [ ]:

for day,flag_file in enumerate(flag_files[:2]):
    #Opening Data/Flags
    hd_data = HERAData(data_file)    
    hd_model = HERAData(model_file) 
    flags = UVFlag(flag_file)
    flags.select(frequencies = flags.freq_array[(flags.freq_array>=130*1e+6) & (flags.freq_array<=170*1e+6)],
                 times = flags.time_array[3000:4000])
    d_data, d_flags, d_nsamples = hd_data.read()
    m_data, m_flags, m_nsamples = hd_model.read()
    for bl in d_data:
        d_flags[bl] = flags.flag_array.squeeze()
        m_flags[bl] = flags.flag_array.squeeze()
    hd_data.update(flags=d_flags)
    hd_model.update(flags=m_flags)
    hd_data.x_orientation='north'
    hd_model.x_orientation='north'
    hd_data.write_uvh5(f"data_{day}.uvh5", clobber=True)
    hd_model.write_uvh5(f"model_{day}.uvh5", clobber=True)
    del d_data,d_flags,d_nsamples,m_data,m_flags,m_nsamples,hd_data,hd_model
    #Calculate Gains/Smooth
    redcal.redcal_run(input_data=f'data_{day}.uvh5', clobber=True, solar_horizon=90, verbose=True)
    abscal.post_redcal_abscal_run(data_file = f'data_{day}.uvh5', redcal_file = f'data_{day}.omni.calfits',
                                  model_files = [f"model_{day}.uvh5"], clobber=True, data_solar_horizon=90,
                                  model_solar_horizon=90)
    cs = smooth_cal.CalibrationSmoother(calfits_list=[f'data_{day}.abs.calfits'])
    cs.time_freq_2D_filter(time_scale=21600)
    cs.write_smoothed_cal(clobber=True,output_replace=('.abs.','.smooth_abs.'))
    #Apply Smooth Gains
    apply_cal.apply_cal(data_infilename=f'data_{day}.uvh5', data_outfilename=f"data_{day}_smoothcal.uvh5",
                        new_calibration=f'data_{day}.smooth_abs.calfits')
    #Fourier-Filter
    vc = vis_clean.VisClean(f"data_{day}_smoothcal.uvh5")
    vc.read()
    vc.vis_clean(standoff=100, min_dly=300, mode='dpss_leastsq', skip_flagged_edge_freqs=True, 
                 flag_rms_outliers=True, max_continuous_flag_channels=1)
    vc.write_filtered_data(filled_outfile_name=f'data_{day}_filtered.uvh5',clobber=True)

File exists; clobbering
File exists; clobbering

Now running redundant calibration without antennas [] ...
Now calibrating ['nn'] polarization(s)...
    Now calibrating times 2457053.0807914445 through 2457053.2052039444 ...

Now saving firstcal gains to data_0.first.calfits
Now saving omnical gains to data_0.omni.calfits
Now saving omnical visibilities to data_0.omni_vis.uvh5


In [9]:
%debug

> /users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_cal/redcal.py(1557)<dictcomp>()
   1555     # spoof autos if none are present in the data
   1556     data_wgts = {bl: predict_noise_variance_from_autos(bl, data, dt=(np.median(np.ediff1d(times_by_bl[bl[:2]]))
-> 1557                                                                      * SEC_PER_DAY))**-1 for bl in data.keys()}
   1558     rv['omni_meta'], omni_sol = rc.omnical(data, log_sol, wgts=data_wgts, conv_crit=oc_conv_crit, maxiter=oc_maxiter,
   1559                                            check_every=check_every, check_after=check_after, gain=gain)

ipdb> print(times_by_bl)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ipdb> print(times_by_bl.values())
ipdb> up
> /users/kshahin/miniconda2/envs/hera3/lib/python3.7/site-packages/hera_cal/redcal.py(1557)redundantly_calibrate()
   1555     # spoof autos if none are present in the data
   1556     data_wgts = {bl: predict_noise_variance_from_autos(bl, data, dt=(np.median(np.ediff1d(times_by_bl[bl[:2]]))
-> 1557                                                                      * SEC_PER_DAY))**-1 for bl in data.keys()}
   1558     rv['omni_meta'], omni_sol = rc.omnical(data, log_sol, wgts=data_wgts, conv_crit=oc_conv_crit, maxiter=oc_maxiter,
   1559                                            check_every=check_every, check_after=check_after, gain=gain)

ipdb> print(times_by_bl.values())


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ipdb> print(times_by_bl.keys())
dict_keys([(0, 1), (0, 2), (0, 3), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 12), (0, 13), (0, 15), (0, 16), (0, 17), (0, 18), (1, 2), (1, 3), (1, 6), (1, 7), (1, 9), (1, 10), (1, 13), (1, 16), (1, 17), (1, 18), (2, 3), (2, 7), (2, 10), (2, 17), (2, 18), (3, 18), (4, 0), (4, 1), (4, 2), (4, 3), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (4, 12), (4, 13), (4, 15), (4, 16), (4, 17), (4, 18), (5, 1), (5, 2), (5, 3), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 12), (5, 13), (5, 16), (5, 17), (5, 18), (6, 2), (6, 3), (6, 7), (6, 9), (6, 10), (6, 13), (6, 17), (6, 18), (7, 3), (7, 10), (7, 18), (8, 1), (8, 2), (8, 3), (8, 6), (8, 7), (8, 9), (8, 10), (8, 13), (8, 16), (8, 17), (8, 18), (9, 2), (9, 3), (9, 7), (9, 10), (9, 17), (9, 18), (10, 3), (10, 18), (11, 0), (11, 1), (11, 2), (11, 3), (11, 5), (11, 6), (11, 7), (11, 8), (11, 9), (11, 10), (11, 12), (11, 13), (11, 15), (11, 16), (11, 17), (11, 18), (12, 1), (12, 2), (12, 3), (12,

In [32]:
hd_data = HERAData(data_file)    
hd_model = HERAData(model_file) 
flags = UVFlag(flag_file)
flags.select(frequencies = flags.freq_array[(flags.freq_array>=130*1e+6) & (flags.freq_array<=170*1e+6)],
                 times = flags.time_array[3000:4000])
d_data, d_flags, d_nsamples = hd_data.read()
m_data, m_flags, m_nsamples = hd_model.read()
print(d_data.keys())

odict_keys([(0, 1, 'xx'), (0, 2, 'xx'), (0, 3, 'xx'), (0, 5, 'xx'), (0, 6, 'xx'), (0, 7, 'xx'), (0, 8, 'xx'), (0, 9, 'xx'), (0, 10, 'xx'), (0, 12, 'xx'), (0, 13, 'xx'), (0, 15, 'xx'), (0, 16, 'xx'), (0, 17, 'xx'), (0, 18, 'xx'), (1, 2, 'xx'), (1, 3, 'xx'), (1, 6, 'xx'), (1, 7, 'xx'), (1, 9, 'xx'), (1, 10, 'xx'), (1, 13, 'xx'), (1, 16, 'xx'), (1, 17, 'xx'), (1, 18, 'xx'), (2, 3, 'xx'), (2, 7, 'xx'), (2, 10, 'xx'), (2, 17, 'xx'), (2, 18, 'xx'), (3, 18, 'xx'), (4, 0, 'xx'), (4, 1, 'xx'), (4, 2, 'xx'), (4, 3, 'xx'), (4, 5, 'xx'), (4, 6, 'xx'), (4, 7, 'xx'), (4, 8, 'xx'), (4, 9, 'xx'), (4, 10, 'xx'), (4, 11, 'xx'), (4, 12, 'xx'), (4, 13, 'xx'), (4, 15, 'xx'), (4, 16, 'xx'), (4, 17, 'xx'), (4, 18, 'xx'), (5, 1, 'xx'), (5, 2, 'xx'), (5, 3, 'xx'), (5, 6, 'xx'), (5, 7, 'xx'), (5, 8, 'xx'), (5, 9, 'xx'), (5, 10, 'xx'), (5, 12, 'xx'), (5, 13, 'xx'), (5, 16, 'xx'), (5, 17, 'xx'), (5, 18, 'xx'), (6, 2, 'xx'), (6, 3, 'xx'), (6, 7, 'xx'), (6, 9, 'xx'), (6, 10, 'xx'), (6, 13, 'xx'), (6, 17, 'xx'), (6,

In [ ]:
#Average Before Filter
calibrated_files = sorted(glob.glob('data_*_smoothcal.uvh5'))
for day,cfile in enumerate(calibrated_files):
    hd = HERAData(cfile)
    data_t,flags_t,nsmaples_t = hd.read()
    if day==0:
        data_avg=DataContainer({bl:data_t[bl]*nsamples_t[bl]*(~flags_t[bl]) for bl in data_t})
        flags_avg =DataContainer({bl:flags_t[bl] for bl in data_t})
        nsamples_avg = DataContainer({bl:nsamples_t[bl]*(~flags_t[bl]) for bl in data_t})
    else:
        for bl in data_avg:
            data_avg[bl] = data_avg[bl] + data_t[bl]*nsamples_t[bl]*(~flags_t[bl])
            flags_avg[bl] = flags_avg[bl] & flags_t[bl]
            nsamples_avg[bl] = nsamples_avg[bl] + nsamples_t[bl]*(~flags_t[bl])
for bl in data_avg:   
    data_avg[bl] = data_avg[bl]/nsamples_avg[bl]
hd.update(data=data_avg, flags=flags_avg, nsamples=nsamples_avg)
hd.write_uvh5('data_avg.uvh5', clobber=True)

In [ ]:
vc = vis_clean.VisClean(f"data_avg.uvh5")
vc.read()
vc.vis_clean(standoff=100, min_dly=300, mode='dpss_leastsq', skip_flagged_edge_freqs=True, 
                 flag_rms_outliers=True, max_continuous_flag_channels=1)
vc.write_filtered_data(filled_outfile_name='data_avg_beforefiltered.uvh5',clobber=True)

In [ ]:
#Average After Filter
filtered_files = sorted(glob.glob('data_*_filtered.uvh5'))
for day,ffiles in enumerate(filtered_files):
    hd = HERAData(ffile)
    data_t,flags_t,nsmaples_t = hd.read()
    if day==0:
        data_avg=DataContainer({bl:data_t[bl]*nsamples_t[bl]*(~flags_t[bl]) for bl in data_t})
        flags_avg =DataContainer({bl:flags_t[bl] for bl in data_t})
        nsamples_avg = DataContainer({bl:nsamples_t[bl]*(~flags_t[bl]) for bl in data_t})
    else:
        for bl in data_avg:
            data_avg[bl] = data_avg[bl] + data_t[bl]*nsamples_t[bl]*(~flags_t[bl])
            flags_avg[bl] = flags_avg[bl] & flags_t[bl]
            nsamples_avg[bl] = nsamples_avg[bl] + nsamples_t[bl]*(~flags_t[bl])
for bl in data_avg:   
    data_avg[bl] = data_avg[bl]/nsamples_avg[bl]
hd.update(data=data_avg, flags=flags_avg, nsamples=nsamples_avg)
hd.write_uvh5('data_avg_afterfiltered.uvh5', clobber=True)


In [25]:
glob.glob('*smooth*')

['HERA_FTsmoothedgains_B.pdf',
 'data_A_smoothcal.uvh5',
 'Day_A_smoothcal.uvh5',
 'HERA_FTsmoothedgains_A.pdf',
 'data_0.smooth_abs.calfits']

In [24]:
cs.write_smoothed_cal(clobber=True,output_replace=('.abs.','.smooth_abs.'))

Mean of empty slice
